In [ ]:
import ast
import numpy as np
import pandas as pd

import torch
# Import all libraries
import pandas as pd
import numpy as np
import re

# Huggingface transformers
import transformers
from transformers import BertModel,BertTokenizer,AdamW, get_linear_schedule_with_warmup

import torch
from torch import nn ,cuda
from torch.utils.data import DataLoader,Dataset,RandomSampler, SequentialSampler

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

#handling html data
from bs4 import BeautifulSoup

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
%matplotlib inline

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)


device(type='cuda')

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [ ]:
df = pd.read_csv('final_dataset.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191253 entries, 0 to 191252
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   CELEX           191253 non-null  object
 1   Text            191253 non-null  object
 2   labels          191253 non-null  object
 3   ohe_labels      191253 non-null  object
 4   gensim_summary  191253 non-null  object
 5   t5_summary      191253 non-null  object
dtypes: object(6)
memory usage: 8.8+ MB


In [ ]:
df.head()

,CELEX,Text,labels,ohe_labels,gensim_summary,t5_summary
0,21980D1231(03),21980D1231(03) Decision No 3/80 of the EEC-Ice...,Greece|agreement (EU)|accession to the Europea...,"[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",21980D1231(03) Decision No 3/80 of the EEC-Ice...,Article 23 (1) shall be amended by the additio...
1,21986A1115(03),15.11.1986 EN Official Journal of the European...,Portugal|protocol to an agreement|accession to...,"[0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",15.11.1986 EN Official Journal of the European...,15.11.1986 EN Official Journal of the European...
2,21987A0720(02),20.7.1987 EN Official Journal of the European ...,protocol to an agreement|revision of an agreement,"[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...",20.7.1987 EN Official Journal of the European ...,20.7.1987 EN Official Journal of the European ...
3,21987D0411(04),21987D0411(04) Decision No 3/86 of the EEC-Swe...,originating product,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0020 - 0024 DECISION N° 3/86 OF THE EEC-SWEDEN...,referred to in paragraph 2: 'The importer's de...
4,21987D0411(05),21987D0411(05) Decision No 3/86 of the EEC-Swi...,originating product,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0026 - 0030 DECISION N° 3/86 OF THE EEC-AUSTRI...,referred to in paragraph 2: 'the exporter's de...


In [ ]:
X = df['Text'].values
X_gensim = df['gensim_summary'].values
X_t5 = df['t5_summary'].values

y = np.asarray([[int(val) for val in ohe[1:-1].split(',')] for ohe in df['ohe_labels'].values])

In [ ]:
from sklearn.model_selection import train_test_split
# First Split for Train and Test
x_train,x_test, x_gensim_train, x_gensim_test, x_t5_train,x_t5_test, y_train,y_test = train_test_split(X, X_gensim, X_t5, y, test_size=0.2, random_state=101,shuffle=True)
# Next split Train in to training and validation
x_tr, x_val, x_gensim_tr, x_gensim_val, x_t5_tr, x_t5_val, y_tr, y_val = train_test_split(x_train, x_gensim_train, x_t5_train, y_train, test_size=0.2, random_state=101,shuffle=True)

In [ ]:
x_tr.shape ,x_val.shape, x_test.shape

((122401,), (30601,), (38251,))

In [ ]:
y_tr.shape, y_val.shape, y_test.shape

((122401, 91), (30601, 91), (38251, 91))

In [ ]:
class QTagDataset (Dataset):
    def __init__(self,quest,tags, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.text = quest
        self.labels = tags
        self.max_len = max_len
        
    def __len__(self):
        return len(self.text)
    
    def __getitem__(self, item_idx):
        text = self.text[item_idx]
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True, # Add [CLS] [SEP]
            max_length= self.max_len,
            padding = 'max_length',
            return_token_type_ids= False,
            return_attention_mask= True, # Differentiates padded vs normal token
            truncation=True, # Truncate data beyond max length
            return_tensors = 'pt' # PyTorch Tensor format
          )
        
        input_ids = inputs['input_ids'].flatten()
        attn_mask = inputs['attention_mask'].flatten()
        #token_type_ids = inputs["token_type_ids"]
        
        return {
            'input_ids': input_ids ,
            'attention_mask': attn_mask,
            'label': torch.tensor(self.labels[item_idx], dtype=torch.float)
            
        } 

In [ ]:
class QTagDataModule (pl.LightningDataModule):
    
    def __init__(self,x_tr,y_tr,x_val,y_val,x_test,y_test,tokenizer,batch_size=16,max_token_len=200):
        super().__init__()
        self.tr_text = x_tr
        self.tr_label = y_tr
        self.val_text = x_val
        self.val_label = y_val
        self.test_text = x_test
        self.test_label = y_test
        self.tokenizer = tokenizer
        self.batch_size = batch_size
        self.max_token_len = max_token_len

    def setup(self):
        self.train_dataset = QTagDataset(quest=self.tr_text, tags=self.tr_label, tokenizer=self.tokenizer,max_len = self.max_token_len)
        self.val_dataset  = QTagDataset(quest=self.val_text,tags=self.val_label,tokenizer=self.tokenizer,max_len = self.max_token_len)
        self.test_dataset  = QTagDataset(quest=self.test_text,tags=self.test_label,tokenizer=self.tokenizer,max_len = self.max_token_len)
        
        
    def train_dataloader(self):
        return DataLoader (self.train_dataset,batch_size = self.batch_size,shuffle = True , num_workers=32)

    def val_dataloader(self):
        return DataLoader (self.val_dataset,batch_size= 32)

    def test_dataloader(self):
        return DataLoader (self.test_dataset,batch_size= 32)

In [ ]:
# Initialize the Bert tokenizer
BERT_MODEL_NAME = "bert-base-cased" # we will use the BERT base model(the smaller one)
Bert_tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME)

In [ ]:
# Initialize the parameters that will be use for training
N_EPOCHS = 12
BATCH_SIZE = 8
MAX_LEN = 512
LR = 2e-05

In [ ]:
# Instantiate and set up the data_module
QTdata_module = QTagDataModule(x_tr,y_tr,x_val,y_val,x_test,y_test,Bert_tokenizer,BATCH_SIZE, MAX_LEN)
QTdata_module.setup()

In [ ]:
class QTagClassifier(pl.LightningModule):
    # Set up the classifier
    def __init__(self, n_classes=91, steps_per_epoch=None, n_epochs=3, lr=2e-5 ):
        super().__init__()

        self.bert = BertModel.from_pretrained(BERT_MODEL_NAME, return_dict=True)
        self.classifier = nn.Linear(self.bert.config.hidden_size, n_classes) # outputs = number of labels
        self.steps_per_epoch = steps_per_epoch
        self.n_epochs = n_epochs
        self.lr = lr
        self.criterion = nn.BCEWithLogitsLoss()
#         self.log = 
        
    def forward(self,input_ids, attn_mask):
        output = self.bert(input_ids = input_ids ,attention_mask = attn_mask)
        output = self.classifier(output.pooler_output)
                
        return output
    
    
    def training_step(self,batch,batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['label']
        
        outputs = self(input_ids,attention_mask)
        loss = self.criterion(outputs,labels)
#         self.log('train_loss',loss , prog_bar=True,logger=True)
        
        return {"loss" :loss, "predictions":outputs, "labels": labels }


    def validation_step(self,batch,batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['label']
        
        outputs = self(input_ids,attention_mask)
        loss = self.criterion(outputs,labels)
#         self.log('val_loss',loss , prog_bar=True,logger=True)
        
        return loss

    def test_step(self,batch,batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['label']
        
        outputs = self(input_ids,attention_mask)
        loss = self.criterion(outputs,labels)
#         self.log('test_loss',loss , prog_bar=True,logger=True)
        
        return loss
    
    
    def configure_optimizers(self):
        optimizer = AdamW(self.parameters() , lr=self.lr)
        warmup_steps = self.steps_per_epoch//3
        total_steps = self.steps_per_epoch * self.n_epochs - warmup_steps

        scheduler = get_linear_schedule_with_warmup(optimizer,warmup_steps,total_steps)

        return [optimizer], [scheduler]

In [ ]:
# Instantiate the classifier model
steps_per_epoch = len(x_tr)//BATCH_SIZE
model = QTagClassifier(n_classes=91, steps_per_epoch=steps_per_epoch,n_epochs=N_EPOCHS,lr=LR)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
#Initialize Pytorch Lightning callback for Model checkpointing

# saves a file like: input/QTag-epoch=02-val_loss=0.32.ckpt
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss', # monitored quantity
    filepath="results/QTag-{epoch:02d}-{val_loss:.2f}",
    save_top_k=3, #  save the top 3 models
    mode='min', # mode of the monitored quantity  for optimization
)

In [ ]:
# Instantiate the Model Trainer
trainer = pl.Trainer(max_epochs = N_EPOCHS , gpus = [0], checkpoint_callback=checkpoint_callback, progress_bar_refresh_rate = 30)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0,1,2,3]


In [ ]:
# Train the Classifier Model
trainer.fit(model, QTdata_module)

/home/ruslan/anaconda3/envs/lformer/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
/home/ruslan/anaconda3/envs/lformer/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given
  warnings.warn(*args, **kwargs)

  | Name       | Type              | Params
-------------------------------------------------
0 | bert       | BertModel         | 108 M 
1 | classifier | Linear            | 69 K  
2 | criterion  | BCEWithLogitsLoss | 0     
/home/ruslan/anaconda3/envs/lformer/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: The dataloader, val dataloader 0, d

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Saving latest checkpoint..


1

In [ ]:
# Evaluate the model performance on the test dataset
trainer.test(model,datamodule=QTdata_module)

/home/ruslan/anaconda3/envs/lformer/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------


1

In [ ]:
# Visualize the logs using tensorboard.
%load_ext tensorboard
%tensorboard --logdir lightning_logs/

# Evaluate Model Performance on Test Set

In [1]:
# Retreive the checkpoint path for best model
model_path = checkpoint_callback.best_model_path

NameError: ignored

In [ ]:
len(y_test), len(x_test)

(38251, 38251)

In [ ]:
# Size of Test set
print(f'Number of Text = {len(x_test)}')

Number of Text = 38251


## setup test dataset for BERT

In [ ]:
from torch.utils.data import TensorDataset

# Tokenize all questions in x_test
input_ids = []
attention_masks = []


for quest in x_test:
    encoded_quest =  Bert_tokenizer.encode_plus(
                    quest,
                    None,
                    add_special_tokens=True,
                    max_length= MAX_LEN,
                    padding = 'max_length',
                    return_token_type_ids= False,
                    return_attention_mask= True,
                    truncation=True,
                    return_tensors = 'pt'      
    )
    
    # Add the input_ids from encoded question to the list.    
    input_ids.append(encoded_quest['input_ids'])
    # Add its attention mask 
    attention_masks.append(encoded_quest['attention_mask'])
    
# Now convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(y_test)

# Set the batch size.  
TEST_BATCH_SIZE = 64  

# Create the DataLoader.
pred_data = TensorDataset(input_ids, attention_masks, labels)
pred_sampler = SequentialSampler(pred_data)
pred_dataloader = DataLoader(pred_data, sampler=pred_sampler, batch_size=TEST_BATCH_SIZE)
    

In [ ]:
pred_data[0]

(tensor([  101,  1853,   119,   128,   119,  1349,   142,  2249,  9018,  3603,
          1104,  1103,  1735,  1913,   149, 21764,   120,  1275, 18732, 25290,
          6258, 13882, 11414,   146, 12347, 17516, 14424, 15681, 15740,   155,
         17020,  2591, 10783, 21669, 11414,   113,  7270,   114,  1302,  5692,
          1559,   120,  1349,  1104,  1743,  1351,  1349,  1113,  1103,  5867,
         10148,  4019,  1106,  1129,  4275,  1107,  2593,  1106,  1103,  1248,
          7597,  8727,  1106,  8886,  1439,  1103,  8886,  1158,  7791,  1533,
          1118,   146, 26318,  1880,  1158, 22575,   113,  7270,   114,  1302,
          5519,  1527,   120,  1349,  7462,  7270, 21564,  2101, 12420,  2249,
         18732, 25290,  6258, 13882, 11414,   117,  5823,  7328,  1106,  1103,
          6599,  1113,  1103, 16068,  5796,  1158,  1104,  1103,  1735,  1913,
           117,  5823,  7328,  1106,  1761, 22575,   113, 16028,   114,  1302,
         13414,  1527,   120,  1384,  1104,  1659,  

In [ ]:
len(pred_dataloader.dataset)

38251

## Prediction on test set

In [ ]:
flat_pred_outs = 0
flat_true_labels = 0

In [ ]:
# Put model in evaluation mode
model = model.to(device) # moving model to cuda
model.eval()

# Tracking variables 
pred_outs, true_labels = [], []
#i=0
# Predict 
for batch in pred_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
  
    # Unpack the inputs from our dataloader
    b_input_ids, b_attn_mask, b_labels = batch
 
    with torch.no_grad():
        # Forward pass, calculate logit predictions
        pred_out = model(b_input_ids,b_attn_mask)
        pred_out = torch.sigmoid(pred_out)
        # Move predicted output and labels to CPU
        pred_out = pred_out.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        #i+=1
        # Store predictions and true labels
        #print(i)
        #print(outputs)
        #print(logits)
        #print(label_ids)
    pred_outs.append(pred_out)
    true_labels.append(label_ids)

In [ ]:
pred_outs[0][0]

array([0.04247205, 0.04140429, 0.03647242, 0.06688445, 0.03912493,
       0.02850365, 0.05135063, 0.03986761, 0.04334361, 0.03944863,
       0.0439194 , 0.03447949, 0.05330167, 0.02522484, 0.03128149,
       0.07561353, 0.0411328 , 0.04793372, 0.02867409, 0.04295354,
       0.0401832 , 0.0490143 , 0.03522522, 0.05312589, 0.06568895,
       0.04104365, 0.05165896, 0.02954815, 0.02608691, 0.05483792,
       0.02598103, 0.05627174, 0.02967295, 0.04997236, 0.0281234 ,
       0.03404609, 0.03096537, 0.04488167, 0.03962629, 0.05882763,
       0.03794998, 0.03045178, 0.02687983, 0.03649532, 0.04703459,
       0.05059582, 0.04397823, 0.03742674, 0.04085401, 0.05194329,
       0.02823091, 0.03924191, 0.06984942, 0.04604209, 0.07216504,
       0.03156662, 0.04187027, 0.03542591, 0.0652856 , 0.03224215,
       0.04741193, 0.05610191, 0.03664237, 0.05110298, 0.03471619,
       0.04287335, 0.02615033, 0.03412688, 0.03657829, 0.06330767,
       0.03689927, 0.05023079, 0.04267962, 0.06344771, 0.03860

In [ ]:
# Combine the results across all batches. 
flat_pred_outs = np.concatenate(pred_outs, axis=0)

# Combine the correct labels for each batch into a single list.
flat_true_labels = np.concatenate(true_labels, axis=0)

In [ ]:
flat_pred_outs.shape , flat_true_labels.shape

((38251, 91), (38251, 91))

## Predictions of Tags in Test set

> The predictions are in terms of logits (probabilities for each of the 16 tags). Hence we need to have a threshold value to convert these probabilities to 0 or 1.

> Let's specify a set of candidate threshold values. We will select the threshold value that performs the best for the test set.

In [ ]:
#define candidate threshold values
threshold  = np.linspace(0.01, 0.11, 10)
threshold

array([0.01      , 0.02111111, 0.03222222, 0.04333333, 0.05444444,
       0.06555556, 0.07666667, 0.08777778, 0.09888889, 0.11      ])

In [ ]:
# convert probabilities into 0 or 1 based on a threshold value
def classify(pred_prob,thresh):
    y_pred = []

    for tag_label_row in pred_prob:
        temp=[]
        for tag_label in tag_label_row:
            if tag_label >= thresh:
                temp.append(1) # Infer tag value as 1 (present)
            else:
                temp.append(0) # Infer tag value as 0 (absent)
        y_pred.append(temp)

    return y_pred

In [ ]:
flat_pred_outs[3]

array([0.04349403, 0.04061484, 0.03730374, 0.06593355, 0.03980355,
       0.02848078, 0.05176379, 0.0393106 , 0.04169189, 0.03947953,
       0.04424963, 0.03469884, 0.05377717, 0.02593262, 0.03130573,
       0.07683558, 0.04121941, 0.04794803, 0.02858874, 0.04369926,
       0.04022895, 0.04783442, 0.0344528 , 0.05290486, 0.06643628,
       0.04216804, 0.05231139, 0.03090497, 0.02609655, 0.05527727,
       0.02642607, 0.05567885, 0.02998942, 0.04910224, 0.0281794 ,
       0.03337159, 0.03059063, 0.0452923 , 0.03958352, 0.05872209,
       0.03924005, 0.02948766, 0.02752234, 0.03639603, 0.04671273,
       0.04988745, 0.04391089, 0.0377197 , 0.0405306 , 0.0507034 ,
       0.02822898, 0.03935443, 0.06961114, 0.04483515, 0.0705504 ,
       0.03214078, 0.04348528, 0.03541722, 0.06648943, 0.032272  ,
       0.04798808, 0.05427981, 0.03614968, 0.05096484, 0.03510578,
       0.04260396, 0.02623725, 0.03380191, 0.03625657, 0.06212581,
       0.03727417, 0.05035603, 0.04253619, 0.06611217, 0.03771

In [ ]:
flat_true_labels[3]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1,
       0, 0, 0])

In [ ]:
from sklearn import metrics
scores=[] # Store the list of f1 scores for prediction on each threshold

#convert labels to 1D array
y_true = flat_true_labels.ravel() 

for thresh in threshold:
    
    #classes for each threshold
    pred_bin_label = classify(flat_pred_outs,thresh) 

    #convert to 1D array
    y_pred = np.array(pred_bin_label).ravel()

    scores.append(metrics.f1_score(y_true,y_pred))

In [ ]:
# find the optimal threshold
opt_thresh = threshold[scores.index(max(scores))]
print(f'Optimal Threshold Value = {opt_thresh}')

Optimal Threshold Value = 0.07666666666666666


## Performance Score Evaluation

In [ ]:
#predictions for optimal threshold
y_pred_labels = classify(flat_pred_outs,opt_thresh)
y_pred = np.array(y_pred_labels).ravel() # Flatten

In [ ]:
print(metrics.classification_report(y_true,y_pred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99   3398632
           1       0.26      0.06      0.09     82209

    accuracy                           0.97   3480841
   macro avg       0.62      0.53      0.54   3480841
weighted avg       0.96      0.97      0.97   3480841



In [ ]:
y = np.asarray([np.asarray(labels.split('|'), dtype='object') for labels in df['labels']], dtype='object')
y_ohe = np.asarray([[int(x) for x in labels[1:-1].split(',')] for labels in df['ohe_labels']])
labellst = []
for i in range(91):    
    llset = set(y[np.where(y_ohe[:, i]==1)][0])
    for ll in y[np.where(y_ohe[:, i]==1)]:
        llset = llset.intersection(ll)
    labellst.append(list(llset)[0])
labellst = np.asarray(labellst)
labellst

array(['Greece', 'agreement (EU)', 'accession to the European Union',
       'originating product', 'Portugal', 'protocol to an agreement',
       'Spain', 'revision of an agreement', 'EU programme',
       'third country', 'European Economic Area', 'fishing area',
       'sea fish', 'fishing rights', 'catch quota',
       'economic concentration', 'labelling', 'approximation of laws',
       'consumer information', 'public health',
       'environmental protection', 'export refund', 'action programme',
       'technical standard', 'international sanctions', 'human rights',
       'import', 'import licence', 'marketing', 'veterinary inspection',
       'agricultural product', 'exchange of information',
       'marketing standard', 'foodstuff', 'consumer protection',
       'cooperation policy', 'air transport', 'EU aid', 'EU financing',
       'EU Member State', 'provision of services',
       'disclosure of information', 'chemical product', 'indemnification',
       'health control', 

In [ ]:
len(labellst)

91

In [ ]:
def inverse_labels(arr):
    inv_labels = []
    for labels in arr:
        inv_labels.append(labellst[np.where(labels)])        
    return np.asarray(inv_labels, dtype='object')

In [ ]:
y_pred = inverse_labels(np.array(y_pred_labels))
y_act = inverse_labels(flat_true_labels)

In [ ]:
y_pred[0]

array([], dtype='<U42')

In [ ]:
df_pred = pd.DataFrame({'Body':x_test,'Actual Tags':y_act,'Predicted Tags':y_pred})
df_pred

,Body,Actual Tags,Predicted Tags
0,29.7.2011 EN Official Journal of the European ...,"[import licence, import (EU)]",[]
1,31.7.2010 EN Official Journal of the European ...,"[State aid, Italy, control of State aid, Europ...",[]
2,17.4.2014 EN Official Journal of the European ...,"[import (EU), common organisation of markets, ...",[]
3,30.3.2015 EN Official Journal of the European ...,"[EU trade mark, trademark law, registered trad...","[economic concentration, merger control]"
4,16.11.2017 EN Official Journal of the European...,"[revision of an agreement, European Economic A...",[]
...,...,...,...
38246,27.3.2004 EN Official Journal of the European ...,[EU financing],[economic concentration]
38247,32001R2522 Commission Regulation (EC) No 2522/...,"[third country, export refund, award of contract]",[]
38248,5.2.2018 EN Official Journal of the European U...,"[European trademark, trademark law, registered...","[economic concentration, merger control]"
38249,"EUROPEAN COMMISSION Brussels,14.6.2018 COM(201...",[EU aid],[]


In [ ]:
df_pred.to_csv('orig_pred.csv', index = False)

In [ ]:
x_test_unique, x_test_idx = np.unique(x_test, return_index = True)

In [ ]:
Ids = df.iloc[df[df['Text'].isin(x_test_unique)]['Text'].drop_duplicates().index]['CELEX']

# DataFrame for Top-5 predictions

In [ ]:
predictions = pd.DataFrame(columns = labellst, data = flat_pred_outs[x_test_idx], index = Ids)
predictions.reset_index(inplace = True)
predictions.rename(columns = {'CELEX': 'id'}, inplace = True)
predictions

,id,Greece,agreement (EU),accession to the European Union,originating product,Portugal,protocol to an agreement,Spain,revision of an agreement,EU programme,...,financial aid,investment company,EU trade mark,equal treatment,European trademark,trademark law,registered trademark,interpretation of the law,action for failure to fulfil an obligation,action for annulment of an EC decision
0,21987D1231(03),0.043090,0.041075,0.036455,0.066226,0.039344,0.028358,0.052039,0.039506,0.042923,...,0.023221,0.035514,0.041078,0.032665,0.041526,0.047570,0.061491,0.046708,0.049267,0.039895
1,21991D1112(07),0.042558,0.041396,0.036473,0.066861,0.038860,0.028433,0.051515,0.039633,0.043302,...,0.023350,0.035643,0.042280,0.032581,0.041549,0.047284,0.061327,0.046805,0.049449,0.039436
2,21994D1231(13),0.042556,0.041404,0.036404,0.066691,0.038950,0.028459,0.051546,0.039711,0.043411,...,0.023406,0.035607,0.042441,0.032667,0.041732,0.047351,0.061190,0.046656,0.049450,0.039392
3,21997D0327(03),0.042257,0.041534,0.036445,0.066839,0.038921,0.028389,0.051424,0.039669,0.043587,...,0.023403,0.035733,0.042804,0.032549,0.042062,0.047117,0.061196,0.046764,0.049602,0.039037
4,21997D0710(18),0.042742,0.041096,0.036478,0.066275,0.039237,0.028385,0.051821,0.039535,0.043032,...,0.023315,0.035577,0.041641,0.032507,0.041606,0.047428,0.061521,0.046637,0.049413,0.039668
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38246,E2010C0159,0.042199,0.042631,0.036613,0.067551,0.038382,0.028703,0.051322,0.039578,0.044408,...,0.023146,0.035646,0.043237,0.032787,0.041428,0.046497,0.061694,0.046580,0.049846,0.039726
38247,E2010C1209(02),0.041952,0.042513,0.037003,0.067960,0.038664,0.028927,0.051467,0.039790,0.044309,...,0.023341,0.035925,0.043402,0.032806,0.041907,0.046747,0.061411,0.046934,0.049732,0.039419
38248,E2012C0112(03),0.042301,0.042232,0.036626,0.067791,0.038849,0.028654,0.051702,0.039570,0.044208,...,0.023214,0.035624,0.042899,0.032722,0.041600,0.046627,0.061485,0.046659,0.049653,0.039546
38249,E2013C0214(02),0.042503,0.041516,0.036407,0.067119,0.039103,0.028581,0.051546,0.039619,0.043593,...,0.023118,0.035673,0.041823,0.032603,0.041429,0.047244,0.061799,0.046647,0.049459,0.039737


In [ ]:
predictions.set_index('id', inplace = True)
predictions.T

id,21987D1231(03),21991D1112(07),21994D1231(13),21997D0327(03),21997D0710(18),21999A0716(01),22000P0301(14),22003D0041,22007D0137,22010A0622(03),...,C2017/301/03,C2019/207/02,C2019/373/03,E2002C0263,E2004C0305R(01),E2010C0159,E2010C1209(02),E2012C0112(03),E2013C0214(02),E2019P0003
Greece,0.043090,0.042558,0.042556,0.042257,0.042742,0.042437,0.043546,0.043528,0.043183,0.042472,...,0.042604,0.042355,0.042523,0.042620,0.042436,0.042199,0.041952,0.042301,0.042503,0.043101
agreement (EU),0.041075,0.041396,0.041404,0.041534,0.041096,0.041867,0.040858,0.040563,0.040671,0.041358,...,0.042102,0.042115,0.041889,0.041925,0.041977,0.042631,0.042513,0.042232,0.041516,0.040614
accession to the European Union,0.036455,0.036473,0.036404,0.036445,0.036478,0.036778,0.036532,0.037060,0.036775,0.036495,...,0.036562,0.036749,0.036514,0.036475,0.036420,0.036613,0.037003,0.036626,0.036407,0.036797
originating product,0.066226,0.066861,0.066691,0.066839,0.066275,0.067171,0.065624,0.065880,0.065673,0.066755,...,0.067262,0.067873,0.067290,0.067570,0.067242,0.067551,0.067960,0.067791,0.067119,0.066071
Portugal,0.039344,0.038860,0.038950,0.038921,0.039237,0.038744,0.039671,0.040024,0.039398,0.038970,...,0.038846,0.038827,0.038768,0.038884,0.038868,0.038382,0.038664,0.038849,0.039103,0.039571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
trademark law,0.047570,0.047284,0.047351,0.047117,0.047428,0.047615,0.048115,0.048928,0.048189,0.047296,...,0.047131,0.047144,0.046979,0.047376,0.046954,0.046497,0.046747,0.046627,0.047244,0.047982
registered trademark,0.061491,0.061327,0.061190,0.061196,0.061521,0.061375,0.061640,0.061640,0.061554,0.061264,...,0.061561,0.061660,0.061637,0.061611,0.061494,0.061694,0.061411,0.061485,0.061799,0.061562
interpretation of the law,0.046708,0.046805,0.046656,0.046764,0.046637,0.046653,0.046684,0.047006,0.046810,0.046698,...,0.046626,0.046827,0.046661,0.046779,0.046568,0.046580,0.046934,0.046659,0.046647,0.046706
action for failure to fulfil an obligation,0.049267,0.049449,0.049450,0.049602,0.049413,0.049853,0.049045,0.049286,0.049283,0.049583,...,0.049540,0.049562,0.049507,0.049630,0.049439,0.049846,0.049732,0.049653,0.049459,0.049420


In [ ]:
pred_transform = predictions.T.copy()
pred_transform

id,21987D1231(03),21991D1112(07),21994D1231(13),21997D0327(03),21997D0710(18),21999A0716(01),22000P0301(14),22003D0041,22007D0137,22010A0622(03),...,C2017/301/03,C2019/207/02,C2019/373/03,E2002C0263,E2004C0305R(01),E2010C0159,E2010C1209(02),E2012C0112(03),E2013C0214(02),E2019P0003
Greece,0.043090,0.042558,0.042556,0.042257,0.042742,0.042437,0.043546,0.043528,0.043183,0.042472,...,0.042604,0.042355,0.042523,0.042620,0.042436,0.042199,0.041952,0.042301,0.042503,0.043101
agreement (EU),0.041075,0.041396,0.041404,0.041534,0.041096,0.041867,0.040858,0.040563,0.040671,0.041358,...,0.042102,0.042115,0.041889,0.041925,0.041977,0.042631,0.042513,0.042232,0.041516,0.040614
accession to the European Union,0.036455,0.036473,0.036404,0.036445,0.036478,0.036778,0.036532,0.037060,0.036775,0.036495,...,0.036562,0.036749,0.036514,0.036475,0.036420,0.036613,0.037003,0.036626,0.036407,0.036797
originating product,0.066226,0.066861,0.066691,0.066839,0.066275,0.067171,0.065624,0.065880,0.065673,0.066755,...,0.067262,0.067873,0.067290,0.067570,0.067242,0.067551,0.067960,0.067791,0.067119,0.066071
Portugal,0.039344,0.038860,0.038950,0.038921,0.039237,0.038744,0.039671,0.040024,0.039398,0.038970,...,0.038846,0.038827,0.038768,0.038884,0.038868,0.038382,0.038664,0.038849,0.039103,0.039571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
trademark law,0.047570,0.047284,0.047351,0.047117,0.047428,0.047615,0.048115,0.048928,0.048189,0.047296,...,0.047131,0.047144,0.046979,0.047376,0.046954,0.046497,0.046747,0.046627,0.047244,0.047982
registered trademark,0.061491,0.061327,0.061190,0.061196,0.061521,0.061375,0.061640,0.061640,0.061554,0.061264,...,0.061561,0.061660,0.061637,0.061611,0.061494,0.061694,0.061411,0.061485,0.061799,0.061562
interpretation of the law,0.046708,0.046805,0.046656,0.046764,0.046637,0.046653,0.046684,0.047006,0.046810,0.046698,...,0.046626,0.046827,0.046661,0.046779,0.046568,0.046580,0.046934,0.046659,0.046647,0.046706
action for failure to fulfil an obligation,0.049267,0.049449,0.049450,0.049602,0.049413,0.049853,0.049045,0.049286,0.049283,0.049583,...,0.049540,0.049562,0.049507,0.049630,0.049439,0.049846,0.049732,0.049653,0.049459,0.049420


In [ ]:
top_5 = pd.DataFrame(index = predictions.index, columns = ['Top_1', 'Top_2', 'Top_3', 'Top_4', 'Top_5', 'labels'])

for col in pred_transform.columns:
    labels = df[df['CELEX'] == col]['labels']
    top_5.loc[col] = list(predictions.T[col].nlargest(5).items()) + [tuple(labels)]
    
top_5

/home/ruslan/anaconda3/envs/lformer/lib/python3.7/site-packages/pandas/core/internals/blocks.py:937: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


,Top_1,Top_2,Top_3,Top_4,Top_5,labels
id,,,,,,
21987D1231(03),"(economic concentration, 0.07606174051761627)","(merger control, 0.0752464160323143)","(Italy, 0.07129500806331635)","(State aid, 0.06973601877689362)","(euro, 0.06711452454328537)","(originating product,)"
21991D1112(07),"(economic concentration, 0.07572702318429947)","(merger control, 0.07539654523134232)","(Italy, 0.07287457585334778)","(State aid, 0.06978736072778702)","(originating product, 0.06686054915189743)","(agreement (EU)|originating product,)"
21994D1231(13),"(economic concentration, 0.07608313858509064)","(merger control, 0.07543617486953735)","(Italy, 0.07328099757432938)","(State aid, 0.07001964747905731)","(originating product, 0.0666908323764801)","(European Economic Area,)"
21997D0327(03),"(economic concentration, 0.07608957588672638)","(merger control, 0.07575301080942154)","(Italy, 0.07385237514972687)","(State aid, 0.07029347121715546)","(originating product, 0.0668390691280365)","(European Economic Area|agreement (EU),)"
21997D0710(18),"(economic concentration, 0.0762624740600586)","(merger control, 0.07563973218202591)","(Italy, 0.07198379188776016)","(State aid, 0.06969413161277771)","(originating product, 0.06627502292394638)",(European Economic Area|public health|environm...
...,...,...,...,...,...,...
E2010C0159,"(economic concentration, 0.07630062848329544)","(merger control, 0.07418380677700043)","(Italy, 0.07382618635892868)","(State aid, 0.07064836472272873)","(originating product, 0.06755146384239197)","(health control,)"
E2010C1209(02),"(economic concentration, 0.07620751857757568)","(merger control, 0.07534019649028778)","(Italy, 0.07374043762683868)","(State aid, 0.07018783688545227)","(originating product, 0.06795982271432877)","(financial aid|control of State aid|State aid,)"
E2012C0112(03),"(economic concentration, 0.07614116370677948)","(merger control, 0.07479923963546753)","(Italy, 0.07311459630727768)","(State aid, 0.07040536403656006)","(originating product, 0.06779059022665024)","(control of State aid|State aid,)"


In [ ]:
top_5.to_csv('orig_top_5_pred.csv')